Library


Main Path


In [ ]:
MAIN_PATH = r""

Konstanta


Random Seed


# Prapemrosesan


# Ekstraksi Fitur


# Dataset


Raw


In [ ]:
df = pd.read_excel(f"{MAIN_PATH}/dataset_labeled.xlsx")

Prapemrosesan


Ekstraksi Fitur (LMS)


Normalisasi, Split, dan Batching


# LGBM


Inisialisasi


In [ ]:
lgbm_model = LGBMClassifier(random_state=21)

training


In [ ]:
# Mau pake grid search, random search, atau optima? atau ga usah aja yak

# CNN


Inisialisasi


In [ ]:
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN, self).__init__()
        test_input = np.random.rand(SAMPLE_RATE * SEGMENT_DURATION)
        lms = get_lms(test_input)
        in_channels = 1
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(256, 256)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, num_classes)
    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.gap(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

training


# Conformer


Inisialisasi


In [ ]:
config = ConformerConfig(
    hidden_size=144,
    num_attention_heads=4,
    intermediate_size=576,
    conv_kernel_size=32,
    num_hidden_layers=8,
    input_feat_per_channel=n_mels,
    input_channels=1,
    max_position_embeddings=time_steps
)

conformer_model = TFConformerModel(config)

input_layer = tf.keras.layers.Input(shape=(n_mels, time_steps))
expand_dim = tf.keras.layers.Reshape((1, n_mels, time_steps))(input_layer)
conv_proj = tf.keras.layers.Conv2D(
    filters=config.hidden_size,
    kernel_size=(3, 3),
    padding='same',
    activation='relu'
)(expand_dim)
squeeze_dim = tf.keras.layers.Reshape((n_mels, time_steps, config.hidden_size))(conv_proj)
conformer_output = conformer_model(squeeze_dim).last_hidden_state
gap = tf.keras.layers.GlobalAveragePooling1D()(conformer_output)
output = tf.keras.layers.Dense(num_classes, activation='softmax')(gap)

conformer_model = tf.keras.Model(inputs=input_layer, outputs=output)

training


# Evaluasi


LGBM


CNN


Conformer
